## Введение в обработку естественного языка

### Урок 12. Модель Transformer-2

Задание

Взять тот же датасет и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует
2. Сделать генерацию заголовков для статьи

In [25]:
!pip install rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
! pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 7.8 MB/s 
     |████████████████████████████████| 432 kB 71.9 MB/s 
     |████████████████████████████████| 6.6 MB 47.8 MB/s 
     |████████████████████████████████| 163 kB 72.6 MB/s 
     |████████████████████████████████| 212 kB 67.9 MB/s 
     |████████████████████████████████| 115 kB 76.2 MB/s 
     |████████████████████████████████| 127 kB 64.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
! pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 7.6 MB/s 


In [10]:
import pandas as pd

from transformers import pipeline

import pprint

import datasets
from datasets import load_dataset
from datasets import load_metric

import random

from IPython.display import display, HTML

In [4]:
summarizer = pipeline("summarization")

summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
"""
)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

[{'summary_text': ' America has changed dramatically during recent years . The number of engineering graduates in the U.S. has declined in traditional engineering disciplines such as mechanical, civil,    electrical, chemical, and aeronautical engineering . Rapidly developing economies such as China and India continue to encourage and advance the teaching of engineering .'}]

In [5]:
summarizer = pipeline("summarization", model='IlyaGusev/mbart_ru_sum_gazeta')

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.47G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

In [6]:
pp = pprint.PrettyPrinter(indent=0, width=100)

In [7]:
summarizer(
    """
    Президент России Владимир Путин предложил полностью отменить большинство плановых проверок российского бизнеса, 
    работа которого не связана с высокими рисками причинения вреда. Об этом российский лидер заявил 
    в ходе выступления на Петербургском международном экономическом форуме (ПМЭФ), трансляцию в своем YouTube-канале ведет «Комсомольская правда».
    «Всем уже давно понятно, нет необходимости ходить и проверять всех подряд. Должен работать риск-ориентированный подход», — отметил он.
    В связи с этим Путин обратился к правительству с поручением определить конкретные параметры подобной реформы проверок бизнеса в ближайшие месяцы.
""",
    max_length=30,
    min_length=10,
)

[{'summary_text': 'Президент России Владимир Путин предложил полностью отменить большинство плановых проверок российского бизнеса, работа которого не связана с высокими рисками причинения'}]

In [8]:
summarizer1 = pipeline("summarization", model='csebuetnlp/mT5_multilingual_XLSum')

Downloading:   0%|          | 0.00/730 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [11]:
raw_datasets = load_dataset("mlsum", 'ru')

raw_datasets

Generating train split:   0%|          | 0/25556 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/750 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/757 [00:00<?, ? examples/s]

Dataset mlsum downloaded and prepared to /root/.cache/huggingface/datasets/mlsum/ru/1.0.0/033c69bbbf1eb198d444f668be75f297cb86251c0671a3d063d1c53c2f231076. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 25556
    })
    validation: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 750
    })
    test: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 757
    })
})

In [12]:
raw_datasets['train'][1]

{'text': 'Буквально за час до боя курантов в подземном переходе станции метро “Охотный Ряд” группа гастарбайтеров из Таджикистана зверски расправилась с 25-летним соплеменником. Как сообщил “МК” руководитель пресс-службы Управления милиции на Московском метрополитене Алексей Мышляев, труп несчастного таджика был обнаружен пассажирами подземки в 22.50. С бедолагой покончили несколькими ножевыми ударами в живот и грудь, ягодицы также были исколоты клинком. Рядом с трупом оперативники обнаружили орудие убийства — обломанное лезвие ножа без ручки. У сыщиков первым делом возникла версия, что зверское убийство совершили скинхеды. Однако вскоре в том же переходе сотрудниками полка милиции по сопровождению поездов были задержаны шестеро уроженцев Таджикистана в возрасте от 20 до 25 лет. А уже 3 января оперативниками криминальной милиции из Управления милиции на Московском метрополитене был задержан седьмой участник расправы над бедолагой. Его нашли в клинике — соплеменник убитого гастарбайтера

In [14]:
metric = load_metric("rouge")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [15]:
def show_random_elements(dataset, num_examples=3):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [16]:
show_random_elements(raw_datasets["train"])

In [17]:
summarizer1(raw_datasets['train']['text'][:10]
    ,
    max_length=200,
    min_length=30,
)

Your max_length is set to 200, but you input_length is only 126. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=63)


[{'summary_text': 'Московская полиция задержала 43-летнего педофила, который в течение трех лет преследовал подростка с ножом.'},
 {'summary_text': 'Московская полиция задержала шестерых участников жестокого убийства таджикского гастарбайтера, который ранее был застрелен в подземном переходе станции метро “Охотный Ряд”.'},
 {'summary_text': 'Президент России Владимир Путин и вице-премьер Дмитрий Медведев приняли участие в акциях в Краснодарском крае.'},
 {'summary_text': 'Накануне празднования Дня святого Валентина в Кунцевском районе Москвы оскорбленный сосед расстрелял своего приятеля и ранил его подругу.'},
 {'summary_text': 'По меньшей мере девять человек погибли в результате аварии в жилом доме No8 в Волгоградской области, где в новогодние праздники спали семьи гостей.'},
 {'summary_text': 'В России вступают в силу изменения в внутренний распорядок дня осужденных в исправительных колониях.'},
 {'summary_text': 'Московский городской суд приговорил к 4 годам колонии общего режима Ал

**Саммари на первый взгляд выглядят неплохо, но имеют мало общего с колонкой  "Summary".**

In [18]:
raw_datasets['train']['summary'][:10]

['Старший преподаватель института коммунального хозяйства и строительства был задержан на днях в Москве за растление школьника',
 'Манежная площадь Москвы стала местом последнего в 2009 году убийства',
 'Президент РФ Дмитрий Медведев с семьей проводит новогодние праздники на горнолыжном курорте “Красная Поляна”, а в воскресенье к нему в гости приехал и премьер Владимир Путин',
 'Первое убийство в 2010 году произошло в Москве на Коцюбинской улице едва ли ни после боя курантов',
 'Шесть человек, в том числе 9-летний ребенок, задохнулись угарным газом в подмосковном Раменском во время новогодних каникул',
 'Хранить все лишние вещи на специальном складе будут теперь российские осужденные',
 'Приговор начальнику отдела по расследованию преступлений против личности УВД по Мытищинскому району Подмосковья Алексею Пономаренко вынесли служители Фемиды',
 'Новый уполномоченный по правам детей начинает войну против педофилов и пиротехники',
 'Грозу всех должников — лучших судебных приставов — тепе

In [19]:
raw_datasets['train']['text'][1]

'Буквально за час до боя курантов в подземном переходе станции метро “Охотный Ряд” группа гастарбайтеров из Таджикистана зверски расправилась с 25-летним соплеменником. Как сообщил “МК” руководитель пресс-службы Управления милиции на Московском метрополитене Алексей Мышляев, труп несчастного таджика был обнаружен пассажирами подземки в 22.50. С бедолагой покончили несколькими ножевыми ударами в живот и грудь, ягодицы также были исколоты клинком. Рядом с трупом оперативники обнаружили орудие убийства — обломанное лезвие ножа без ручки. У сыщиков первым делом возникла версия, что зверское убийство совершили скинхеды. Однако вскоре в том же переходе сотрудниками полка милиции по сопровождению поездов были задержаны шестеро уроженцев Таджикистана в возрасте от 20 до 25 лет. А уже 3 января оперативниками криминальной милиции из Управления милиции на Московском метрополитене был задержан седьмой участник расправы над бедолагой. Его нашли в клинике — соплеменник убитого гастарбайтера попал на

**Сгенерируем заголовки.**

In [20]:
summarizer(raw_datasets['train']['text'][:10]
    ,
    max_length=30,
    min_length=10,
)

[{'summary_text': 'В Москве задержали педофила, который в течение трех лет преследовал подростка в надежде совратить его'},
 {'summary_text': 'В подземном переходе станции метро «Охотный Ряд» группа гастарбайтеров из Таджикистана зверски рас'},
 {'summary_text': 'Президент России Владимир Путин расплатился за горнолыжные путешествия с вице-премьером Дмитрием Козаком и'},
 {'summary_text': 'Сосед расстрелял соседа из-за претензий по поводу затопления квартиры. Накануне праздника сосед'},
 {'summary_text': 'В Волгоградской области в жилом доме задымился отопительный агрегат. В результате ЧП погибли пять человек, в том числе'},
 {'summary_text': 'На воспитанниках колоний для несовершеннолетних будут выделять после ужина не меньше 30 минут на их личное время. В'},
 {'summary_text': 'В Мытищах приговорен к четырем годам колонии общего режима Алексей Пономаренко, обвиняемый в похище'},
 {'summary_text': 'Президент России Дмитрий Медведев назначил известного адвоката и шоумена Павла Астахова 

**Сгенерируем еще раз.**

In [21]:
summarizer1(raw_datasets['train']['text'][:10]
    ,
    max_length=30,
    min_length=10,
)

[{'summary_text': 'Московская полиция задержала 43-летнего педофила, который в течение трех лет преследовал подростка с нож'},
 {'summary_text': 'Московская полиция задержала шестерых участников жестокого убийства таджикского гастарбайтера, который'},
 {'summary_text': 'Президент России Владимир Путин и вице-премьер Дмитрий Медведев приняли участие в акциях в Краснодарском крае.'},
 {'summary_text': 'Накануне празднования Дня святого Валентина в Кунцевском районе Москвы оскорбленный сосед расстрелял'},
 {'summary_text': 'По меньшей мере девять человек погибли в результате аварии в жилом доме No8 в Волгоградской области.'},
 {'summary_text': 'В России вступают в силу изменения в правила внутреннего распорядка колоний для несовершеннолетних.'},
 {'summary_text': 'Московский городской суд приговорил к 4 годам колонии общего режима Алексея Пономаренко, который незаконно'},
 {'summary_text': 'Уполномоченный по правам детей в России Павел Астахов заявил в интервью российскому телеканалу “МК'

**Данные модели плохо подходят для генерации заголовков, так как часть слов теряется.**

In [22]:
raw_datasets['train']['summary'][:10]

['Старший преподаватель института коммунального хозяйства и строительства был задержан на днях в Москве за растление школьника',
 'Манежная площадь Москвы стала местом последнего в 2009 году убийства',
 'Президент РФ Дмитрий Медведев с семьей проводит новогодние праздники на горнолыжном курорте “Красная Поляна”, а в воскресенье к нему в гости приехал и премьер Владимир Путин',
 'Первое убийство в 2010 году произошло в Москве на Коцюбинской улице едва ли ни после боя курантов',
 'Шесть человек, в том числе 9-летний ребенок, задохнулись угарным газом в подмосковном Раменском во время новогодних каникул',
 'Хранить все лишние вещи на специальном складе будут теперь российские осужденные',
 'Приговор начальнику отдела по расследованию преступлений против личности УВД по Мытищинскому району Подмосковья Алексею Пономаренко вынесли служители Фемиды',
 'Новый уполномоченный по правам детей начинает войну против педофилов и пиротехники',
 'Грозу всех должников — лучших судебных приставов — тепе

In [23]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

**Произведем расчет метрики rouge.**

In [24]:
preds = summarizer1(raw_datasets['train']['text'][:10],
                    max_length=25,
                    min_length=10,
                    )
summary = raw_datasets['train']['summary'][:10]
true_labels = [x.lower() for x in summary]
metric.compute(predictions=preds, references=true_labels)

{'rouge1': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)),
 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)),
 'rougeL': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)),
 'rougeLsum': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0))}

**В целом суммаризация выдает весьма осмысленные тексты, но для генерации заголовков подходит плохо.**